# Language Models

## Copyright notice

This version (c) 2019 Fabian Offert, [MIT License](LICENSE). 

## Learn more

- Understanding LSTM Networks: http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- Karpathy's char-rnn: https://github.com/karpathy/char-rnn

## Imports

We are using the Gensim and SpaCy NLP libraries that provide high-level interfaces for a lot of common NLP tasks, in addition to some basic system librarie.

In [5]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np

import gensim # conda install gensim
import spacy # conda install -c conda-forge spacy

import string
import os
import random
from collections import *
import math

nlp = spacy.load('en') # python -m spacy download en

## Sources

You should have received the Proust dataset by email. Additionally, the Shakespeare dataset that Andrej Karpathy uses in his article can be downloaded from his site: http://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt . The pre-trained Google News word embeddings can be downloaded from [the original word2vec site](https://code.google.com/archive/p/word2vec/).

## Word Embeddings

- Word2vec is a particularly computationally-efficient predictive model for learning word embeddings from raw text.
- What is being learned? The probability of a word appearing in a fixed window around another word.
- The neural network is trained on word pairs, but the hidden weights are actually used.
- Two flavors: bag-of-words, skip-gram
- Works well with very large corpora (109+ words)
- Can be used to predict structural mappings, i.e. analogies. “Classic” task: man is to king = woman is to?

![](img/embeddings.jpg)

### Pre-trained embeddings (Google News corpus, $10^{10}$ words)

Using pre-trained embeddings with Gensim is as simple as one line of code. See the [Gensim word2vec documentation](https://radimrehurek.com/gensim/models/word2vec.html) for a list integrated vector operations.

In [395]:
# C binary format
wv_news = gensim.models.KeyedVectors.load_word2vec_format('sources/google300.bin', binary=True)  

The format that Gensim requires for analogy questions is a bit confusing. It is derived from the actual (arithmetical) vector operation, where `king - man + woman = ?`, which is why woman and king are the "positive" terms and man is the "negative" term. Below, we transcribe it to more intuitive variables.

In [422]:
a = 'man'
# is to
b = 'king'
# like 
c = 'woman'
# is to ?

wv_news.wv.most_similar(positive=[c, b], negative=[a])

[('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321243286133),
 ('kings', 0.5236844420433044),
 ('Queen_Consort', 0.5235945582389832),
 ('queens', 0.518113374710083),
 ('sultan', 0.5098593235015869),
 ('monarchy', 0.5087411999702454)]

We can use the wonderful [TensorBoard embedding projector](https://projector.tensorflow.org/) to visualize the embeddings

In [ ]:
dim = 300
words = []

vectors = np.zeros((len(wv_news.wv.vocab), dim))
for i, word in enumerate(wv_news.wv.index2word):
    vectors[i] = wv_news.wv[word]
    words.append(word)

with open('data.tsv', 'w+') as f:
    for vector in vectors.tolist():
        for point in vector:
            f.write(str(point) + '\t')
        f.write('\n')

with open('metadata.tsv', 'w+') as f:
    for word in words:
        f.write(word + '\n')

### Self-trained embeddings ("In Search of Lost Time")

"In Search of Lost Time" is all about [links and similarities](https://en.wikipedia.org/wiki/Involuntary_memory): between times, places, things, senses, and people. Its arguably most famous scene is the "Madeleine" passage, where the experience of eating a simple [French coffee cake](https://en.wikipedia.org/wiki/Madeleine_(cake)) leads the narrator to remember a childhood episode and, subsequently, his whole childhood and youth. How can we explore these links and similarities computationally? With word embeddings, of course.

![](https://marimann.files.wordpress.com/2012/02/proust_16anni_nadar.jpg)

> And suddenly the memory returns. The taste was that of the little crumb of madeleine which on Sunday mornings at Combray (because on those mornings I did not go out before church-time), when I went to say good day to her in her bedroom, my aunt Leonie used to give me, dipping it first in her own cup of real or of lime-flower tea. The sight of the little madeleine had recalled nothing to my mind before I tasted it; perhaps because I had so often seen such things in the interval, without tasting them, on the trays in pastry-cooks' windows, that their image had dissociated itself from those Combray days to take its place among others more recent; perhaps because of those memories, so long abandoned and put out of mind, nothing now survived, everything was scattered; the forms of things, including that of the little scallop-shell of pastry, so richly sensual under its severe, religious folds, were either obliterated or had been so long dormant as to have lost the power of expansion which would have allowed them to resume their place in my consciousness. But when from a long-distant past nothing subsists, after the people are dead, after the things are broken and scattered, still, alone, more fragile, but with more vitality, more unsubstantial, more persistent, more faithful, the smell and taste of things remain poised a long time, like souls, ready to remind us, waiting and hoping for their moment, amid the ruins of all the rest; and bear unfaltering, in the tiny and almost impalpable drop of their essence, the vast structure of recollection.
And once I had recognized the taste of the crumb of madeleine soaked in her decoction of lime-flowers which my aunt used to give me (although I did not yet know and must long postpone the discovery of why this memory made me so happy) immediately the old grey house upon the street, where her room was, rose up like the scenery of a theatre to attach itself to the little pavilion, opening on to the garden, which had been built out behind it for my parents (the isolated panel which until that moment had been all that I could see); and with the house the town, from morning to night and in all weathers, the Square where I was sent before luncheon, the streets along which I used to run errands, the country roads we took when it was fine. And just as the Japanese amuse themselves by filling a porcelain bowl with water and steeping in it little crumbs of paper which until then are without character or form, but, the moment they become wet, stretch themselves and bend, take on colour and distinctive shape, become flowers or houses or people, permanent and recognisable, so in that moment all the flowers in our garden and in M. Swann's park, and the water-lilies on the Vivonne and the good folk of the village and their little dwellings and the parish church and the whole of Combray and of its surroundings, taking their proper shapes and growing solid, sprang into being, town and gardens alike, from my cup of tea.

Gensim thankfully allows us to pass a Python generator as input to the word2vec model, which allows us to read the very large corpus sentence by sentence, instead of reading it into memory at once.

In [5]:
class yield_file(object):
    def __init__(self, filename):
        self.filename = filename
 
    # By default, these yields nice, clean lists of sentence words
    def __iter__(self):
        # File only has line brakes at paragraph boundaries
        # Always remove possible BOMs with vim -c "set nobomb" -c wq! myfile
        for paragraph in open(self.filename):
            for sentence in paragraph.split('.'):
                
                # Use only lower case
                sentence = sentence.lower()

                # Remove all punctuation
                exclude = set(string.punctuation)
                sentence = ''.join(char for char in sentence if char not in exclude)

                # Remove whitespaces
                sentence = sentence.strip()

                # Line as list
                sentence = sentence.split()
                
                # Only return non-empty lines
                if len(sentence) > 0: yield sentence

Let's see if our model is general enough to answer the standard analogy question

In [6]:
sentences = yield_file('sources/proust_ascii.txt') 
wv_proust = gensim.models.Word2Vec(sentences, size=300, window=5, min_count=5, workers=4)
wv_proust.wv.most_similar(positive=['woman', 'king'], negative=['man'])
# wv_proust.wv.accuracy('7-nlp/questions-words.txt')

[('queen', 0.840269923210144),
 ('laundress', 0.8244017958641052),
 ('historian', 0.82276451587677),
 ('patronage', 0.8200148344039917),
 ('jardin', 0.8188639879226685),
 ('south', 0.808866560459137),
 ('balloon', 0.8062515258789062),
 ('manager', 0.8060844540596008),
 ('painter', 0.803223729133606),
 ('ambassador', 0.799126148223877)]

It is, which is good news for our further investigation of our model.

### Improving Embeddings with named entity recognition

We would like to extract some more semantic information from our Proust model, namely we would like to see if character relations are preserved in vector space. To do that, we will run named entity recognition before building the model. SpaCy comes with a built-in named entity recognizer that we are using below.

In [7]:
class yield_file_tagged(object):
    
    def __init__(self, filename):
        self.filename = filename
    
    def _tag_word(self, word):
        text = word.text
        if word.ent_type_: tag = word.ent_type_
        else: tag = word.pos_
        return text + '|' + tag
 
    # By default, these yields nice, clean lists of sentence words
    def __iter__(self):
        # File only has line brakes at paragraph boundaries
        # Always remove possible BOMs with vim -c "set nobomb" -c wq! myfile
        for paragraph in open(self.filename):
            # SpaCy magic
            doc = nlp(paragraph)
    
            # Detect and merge entitites
            for ent in doc.ents:
                ent.merge(tag=ent.root.tag_, lemma=ent.text, ent_type=ent.root.ent_type_)
    
            # Detect and merge noun chunks
            for nc in doc.noun_chunks:
                while len(nc) > 1 and nc[0].dep_ not in ('advmod', 'amod', 'compound'):
                    nc = nc[1:]
                nc.merge(tag=nc.root.tag_, lemma=nc.text, ent_type=nc.root.ent_type_)
            
            for sentence in doc.sents:
                words = []
                for word in sentence:
                    if not word.is_space: 
                        words.append(self._tag_word(word))
                            
                yield words

In [8]:
sentences = yield_file_tagged('sources/proust_ascii.txt') 
wv_proust_tagged = gensim.models.Word2Vec(sentences, size=300, window=5, min_count=5, workers=4)
wv_proust_tagged.save('wv_proust_tagged.gensimmodel')

Our test case here is a hard one. At the very end of the whole book, it becomes apparent that M. de Charlus had romantic relationships with both Morel and Marcel's friend Bloch. We will investigate if this is preserved in the vector space by checking it against one of the more prominent romantic relations, the relation between Marcel and Albertine.

In [9]:
wv_proust_tagged_reloaded = gensim.models.Word2Vec.load('wv_proust_tagged.gensimmodel')
print(wv_proust_tagged_reloaded)
wv_proust_tagged_reloaded.wv.most_similar(positive=['Albertine|PERSON', 'M. de Charlus|PERSON'], negative=['I|PRON'])

Word2Vec(vocab=10617, size=300, alpha=0.025)


[('Morel|PERSON', 0.8458129167556763),
 ('Bloch|PERSON', 0.8372248411178589),
 ('Saint-Loup|ORG', 0.8185210824012756),
 ('Swann|PERSON', 0.8098665475845337),
 ('M. de Norpois|ORG', 0.8037635087966919),
 ('M. de Guermantes|PERSON', 0.7948862910270691),
 ('Elstir|PERSON', 0.7704919576644897),
 ('Robert|PERSON', 0.765677809715271),
 ('Odette|PROPN', 0.7484169006347656),
 ('Cottard|PERSON', 0.7473478317260742)]

## Information theory and Markov Models

### Information: Hartley/Nyquist/Shannon

Watch: https://www.youtube.com/watch?v=2s3aJfRr9gE

- Quadruplex telegraph: different-strength and different-direction currents: +1V, -1V, +5V, -5V
- Resolution is limited by electrical noise
- Problem: electrical pulses bleed into each other beyond a certain pulse rate: [intersymbol interference](https://en.wikipedia.org/wiki/Intersymbol_interference), for digital systems: [Nyquist–Shannon sampling theorem](https://en.wikipedia.org/wiki/Nyquist%E2%80%93Shannon_sampling_theorem)
- Based on these fundamental physical limitations, we can define the [channel capacity](https://en.wikipedia.org/wiki/Channel_capacity) by means of the possible rate of symbols per unit of time $n$ and the possible differences per symbol $s$
- This generates a decision tree with $s^n$ leaves, where the number of leaves/base of he tree is the size of *message space*
- Given a message composed of symbols from this message space, how many questions per symbol do I have to ask at minimum to guess the content of a message. E.g., for a symbol space of size $26$ (the alphabet), I could ask: "Is it less than N?". If it is: "Is it less than G?", etc.. I will need minimum 4, and maximum 5 questions to be 100% certain of the sent symbol.
- In general, $2^{\text{questions}} = \text{message space}$
- It follows, that, on average, I will need $x = \log_2(26) \approx 4.7$ questions to guess one symbol correctly.
- Hartley (1928): *information* of a mesage $H = n \log2(s)$, where $n$ is the number of symbols in the message, and $s$ is the number of different symbols available

### Dependence/independence: Markov

Watch: https://www.youtube.com/watch?v=WyAtOqfCiBw and https://www.youtube.com/watch?v=Ws63I3F7Moc

- Bernoulli's [law of large numbers](https://en.wikipedia.org/wiki/Law_of_large_numbers): convergence of results of a trial on the [expected value](https://en.wikipedia.org/wiki/Expected_value) as the number of trials approaches infinity
- Generally: for large numbers of random trials, things converge on averages, and the probability of variation away from averages forms predictable distributions ([central limit theorem](https://en.wikipedia.org/wiki/Central_limit_theorem))
- Nekrasov: independent variables are a necessary condition for the law of large numbers
- Markov: law of large numbers also applies for dependent variables, as demonstrated by [Markov chains](https://en.wikipedia.org/wiki/Markov_chain): *states* and *transition matrices* introduce *short-term memory*
- Examples: https://en.wikipedia.org/wiki/Examples_of_Markov_chains

### Approximations to English: Shannon

- Shannon, in "A Mathematical Theory of Communication" (1948), famoulsy defines communication as selection:
> "The fundamental problem of communication is that of reproducing at one point either exactly or approximately a message selected at another point. Frequently the messages have meaning; that is they refer to or are correlated according to some system with certain physical or conceptual entities. These semantic aspects of communication are irrelevant to the engineering problem. The significant aspect is that the actual message is one selected from a set of possible messages. The system must be designed to operate for each possible selection, not just the one which will actually be chosen since this is unknown at the time of design."
- We can model such systems - including a natural language like English - with Markov chains:
> "We can also approximate to a natural language by means of a series of simple artificial languages. The zero-order approximation is obtained by choosing all letters with the same probability and independently. The first-order approximation is obtained by choosing successive letters independently but each letter having the same probability that it has in the natural language. 5 Thus, in the first-order approximation to English, E is chosen with probability. 12 (its frequency in normal English) and W with probability .02, but there is no influence between adjacent letters and no tendency to form the preferred digrams such as TH, ED, etc. In the second-order approximation, digram structure is introduced. After a letter is chosen, the next one is chosen in accordance with the frequencies with which the various letters follow the first one. This requires a table of digram frequencies [...]. In the third-order approximation, trigram structure is introduced. Each letter is chosen with probabilities which depend on the preceding two letters."
- Why does this work? Exactly because the transition probabilities converge, given that we have enough "trials", on a reasonable distribution, from which we can build a transition matrix
- How do we measure the information of such a process? Shannon:
> "We have represented a discrete information source as a Markoff process. Can we define a quantity which will measure, in some sense, how much information is "produced" by such a process, or better, at what rate information
is produced?"
- This measure is [entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory))!
- Entropy is defined as $H = \sum_{i=1}^{n}p_i \log(\frac{1}{p_i})$
- It is based on the *uncertainty* of a fair coin flip: both outcomes are equally likely, the entropy is $0.5 \log(\frac{1}{0.5}) + 0.5 \log(\frac{1}{0.5}) = 0.5+0.5 = 1$. Generally, entropy is maximum where all outcomes are equally likely.

![](img/shannon.jpg)

- This can again be illustrated by the average number of questions I need to ask to find a symbol
- Suppose we have a machine (a "discrete informations source represented as a Markov process") that generates letters from the alphabet A,B,C, and D with $P=(0.5|0.25|0.125|0.125)$. To find the next letter I could ask "Is it A?". Because it is more likely that it actually *is* A, we are done in 50% of the time with just one question. If it is not A, then we can ask: "Is it B?", and we will be right 75% of the time (because we already excluded the possibility of it being A). Again, we build a decision tree, just with "weights", i.e. probabilities, attached to its branches, where the height of the decision tree represents the maximum number of questions we need to ask to find the next letter.
- The total entropy of the source is the entropy of each state times the likelihood of that state.
- Entropy has very important implications for compression. For instance, with [Huffman coding](https://en.wikipedia.org/wiki/Huffman_coding) (lossless compression), the limit of compression is the entropy of the message source.

### Generating Proust from scratch with an unsmoothed maximum likelihood character level language (Markov) model

In [423]:
# Sanity check for entropy calculation: fair coin toss
print(2 * (0.5 * math.log((1/0.5), 2)))

# Sanity check for entropy calculation: zero order, 4-letter alphabet
H = 0
for i in range(4):
    Hij = 0
    Pi = 0.25
    for pij in range(4):
        Hij += 0.25 * math.log((1/0.25), 2)
    H +=  Pi * Hij
print(H)

1.0
2.0


In [442]:
class Markov():
    
    def __init__(self, filename, order=4, word=False, train_test_split=0.1):
        # Read the whole corpus into memory
        self.order = order
        self.word = word
        data = open(filename).read()
        
        # If this is a word model, we will operate on a list - almost all functions still work because
        # strings are just lists of characters!
        if self.word: 
            data = data.split()
        
        # Split in train and test data
        test_data_size = math.floor(len(data)*train_test_split)
        self.train_data = data[:-test_data_size]
        self.test_data = data[-test_data_size:]
        
        self._build_model()
    
    def _build_model(self):
        # Defaultdict of Counter dicts to keep track of transition probabilities
        # Counter dict to keep track of state probabilities
        self._transition_probs = defaultdict(Counter)
        self._state_probs = Counter()
        
        # Create state and transition probability matrices
        for position in range(len(self.train_data)-self.order):
            # Get current state and next state of specified order
            state, next_state = self.train_data[position:position+order], self.train_data[position+order]
            if self.word: 
                state = " ".join(state)
                next_state = " ".join([next_state])
            self._state_probs[state]+=1
            self._transition_probs[state][next_state]+=1
            
        # Store vocabulary
        self.vocabulary = list(self._state_probs.keys())
                
        # Normalize matrices
        self._transition_probs = {state:self._normalize(next_states) for state, next_states in self._transition_probs.items()}
        self._state_probs = self._normalize(self._state_probs)
        
        # Compute entropy: for higher orders, the entropy decreases as the size of the alphabet increases
        # See Shannon (1948), p. 14
        H = 0
        for i in self._transition_probs:
            Hij = 0
            Pi = self._state_probs[i]
            for pij in self._transition_probs[i].values():
                Hij += pij * math.log((1/pij), 2)
            H +=  Pi * Hij
        self.entropy = H
    
    # Helper function to normalize a Counter dictionary w.r.t. its total sum
    def _normalize(self, dictionary):
        sigma = float(sum(dictionary.values()))
        return {key:value/sigma for key, value in dictionary.items()}
    
    def test(self):
        score = 0
        attempts = 0
        for position in range(len(self.test_data)-self.order):
            # Get current state and next state of specified order
            state, next_state = self.test_data[position:position+order], self.test_data[position+order]
            if self.word: 
                state = " ".join(state)
                next_state = " ".join([next_state])
            # There might be unknown stuff in the test data
            if state in self._transition_probs:
                if next_state in self._transition_probs[state]:
                    score += self._transition_probs[state][next_state]
            attempts +=1
        return score/attempts
                
    def generate_ngram(self, history):
        state = history[-self.order:]
        if self.word: state = " ".join(state)

    def generate_text(self, n=1000):
        # Initial state is a random pick from vocabulary
        history = random.choice(self.vocabulary)
        ngrams = []
        for position in range(n):
            
            if self.word:
                state = " ".join(history.split()[-self.order:])
            else:
                state = history[-self.order:]
                
            p = list(self._transition_probs[state].values())
            v = list(self._transition_probs[state].keys())
            ngram = np.random.choice(v, p=p)
            
            if self.word:
                history = " ".join(history.split()[-self.order:] + ngram.split())
            else:
                history = history[-self.order:] + ngram
            
            ngrams.append(ngram)
        if self.word: return " ".join(ngrams)
        else: return "".join(ngrams)

Let's generate different oder Markov models for the Proust dataset. For each order, we compute the source entropy, and test it on 10% of the text we are keeping back.

In [436]:
for order in [1,2,3,4,5,6,7,8,9,10,15,20]:
    mm = Markov('sources/proust_ascii.txt', order, word=False)
    print('### ORDER OF MODEL: ' + str(order) + ' ###')
    print('### ENTROPY/SYMBOL OF MODEL: ' + str(mm.entropy) + ' ###')
    print('### TOTAL VOCABULARY OF MODEL: ' + str(len(mm.vocabulary)) + ' ###')
    print('### TEST SCORE OF MODEL: ' + str(mm.test()) + ', RANDOM BASELINE: ' + str(1/len(mm.vocabulary)) + ' ###')
    print('\n' + mm.generate_text(1000) + '\n')

### ORDER OF MODEL: 1 ###
### ENTROPY/SYMBOL OF MODEL: 3.4182733964359446 ###
### TOTAL VOCABULARY OF MODEL: 77 ###
### TEST SCORE OF MODEL: 0.1636285225010647, RANDOM BASELINE: 0.012987012987012988 ###

9tem ay o ct pof tersey, m w tinouie bs throrr ito. ibacod inod ofrinuders aumey itiched t Lof ssh s ftonermutinthon r; abutily, ang sowhero ned rlbon!" Cough ive innto htom alleshanine, tof tis, tofasty iberad s h I keissoncunoubintivestharup I athe the. t whe desks o I haverlf thitone de ay per ho icand, the; byed t,"Onlly. Mmersuthat woo wevordin'st theded sthere nes, t s). re, f cerenit Gueloureore anghext ind pid Upr cany y, soute I thof ou, wr hitidmat bedut. o ongghit fave f bs ade ur Swan r ted t whenampancism hin sad ust the by, trilowas, thacond tomysisicefrsocour tanche win eron dereporindears hthetofrat s casoure is h t meresinotiad y e one pe th de ll m perof forinethes won mungelot re, whano arechest thofeangeicas in aid thes, spale owas pinthionthe breean bor ff tomermpr

### ORDER OF MODEL: 8 ###
### ENTROPY/SYMBOL OF MODEL: 0.9741601092508716 ###
### TOTAL VOCABULARY OF MODEL: 1540032 ###
### TEST SCORE OF MODEL: 0.4954408249741863, RANDOM BASELINE: 6.493371566305116e-07 ###

gular little harbour, a dry dock, or possible to form a play the gaps between the memory of his dream had given to dare to be irremediable, irremovable.
And now into the shade of a visit them on. But that it was simply had to be shewn less upon the pointed by that she had fallen back upon."
I had been strong gust of window-curtains after she had not be of my grandmother to leave me, I had exclaimed in destroyed. My affection, and that she had come to dine at Feterne family was that this is what excuse to cry, all the story was taking the anguish of Albertine, and that, for that might never, what Francoise, gave a wrong moment, at so late an internationalist orator had just been speak with cold and dishonoured world of interest, will make a 'good impressed. For the possible to ask

An order of 6 seems to offer the best test accuracy. A possible explanation could be that a sixth-order character model is probably roughly equivalent to a first-order word model (i.e. the average word length would be ~6). Note that with very high order models, we can almost deterministically describe the text (i.e. the entropy aproaches zero). This makes intuitive sense, as very high order models will have little redundancy in their vocabulary. As a consequence, their test accuracy is also very low.

In [434]:
order = 30
mm = Markov('sources/proust_ascii.txt', order, word=False)
print('### ENTROPY/SYMBOL OF MODEL: ' + str(mm.entropy) + ' ###')
print('### TOTAL VOCABULARY OF MODEL: ' + str(len(mm.vocabulary)) + ' ###')
print('### TEST SCORE OF MODEL: ' + str(mm.test()) + ', RANDOM BASELINE: ' + str(1/len(mm.vocabulary)) + ' ###')
print('\n' + mm.generate_text(1000))

### ENTROPY/SYMBOL OF MODEL: 0.000919803390158035 ###
### TOTAL VOCABULARY OF MODEL: 6522239 ###
### TEST SCORE OF MODEL: 0.001233485236958012, RANDOM BASELINE: 1.533215817451645e-07 ###

e, of this pride. Almost all the rest sprang from a feeling of which I was then still ignorant, and for which I could not therefore be blamed for not making due allowance. I could at least, failing this unknown element, have mingled with his pride, had I remembered the words of Mme. de Guermantes, a trace of madness. But at that moment the idea of madness never even entered my head. There was in him, according to me, only pride, in me there was only fury. This fury (at the moment when M. de Charlus ceased to shout, in order to refer to his august toes, with a majesty that was accompanied by a grimace, a nausea of disgust at his obscure blasphemers), this fury could contain itself no longer. With an impulsive movement, I wanted to strike something, and, a lingering trace of discernment making me respec

Example with another source, Shakespeare.

In [443]:
order = 5
mm = Markov('sources/shakespeare_input.txt', order, word=False)
print('### ENTROPY/SYMBOL OF MODEL: ' + str(mm.entropy) + ' ###')
print('### TOTAL VOCABULARY OF MODEL: ' + str(len(mm.vocabulary)) + ' ###')
print('### TEST SCORE OF MODEL: ' + str(mm.test()) + ', RANDOM BASELINE: ' + str(1/len(mm.vocabulary)) + ' ###')
print('\n' + mm.generate_text(1000) + '\n')

### ENTROPY/SYMBOL OF MODEL: 1.6259610536894864 ###
### TOTAL VOCABULARY OF MODEL: 264864 ###
### TEST SCORE OF MODEL: 0.4504723860515316, RANDOM BASELINE: 3.775522532318473e-06 ###

rp
Than ye could be friend buzzing none of mean you question in thy throught unto thrice speak well this stiffs on, on me:
A most stars with a good hell out of manhood hearts, and the
princes in Banquo?

Second Citizen:
No way to-night?
Unarm, my lord
Be rust needs is new-dyed be
banner more, our sword quite lost a million for Rosalind, and so prate! Away! away!

CATESBY:
What's moulders.

TIMON:
Come, go: one o' both turmoil
As true.
Still him and yet as Dian!
We burther look pale.
How sit fashion me: lie with you must not my heels.
Even some in.
I descry
A faults; but her her too: he winking wounds are was itself a worthy tongues! you all
Propinquiring upon Pompey, you needs, whom?

BENEDICK:
No, by thee
Than when I utter guidings.

CORNWALL:
You are were that is wealth
'Gainst thou, or is it?

BEATRICE:

# CLLMs and LSTMs

- Now we understand why it is important to consider sequences as Markov models, i.e. take the "history" of a sequence into account: this gives us the possibility to model a sequence as a set of dependend variables, which enables the model to "encode" structure beyond simple symbol ratios. CLLMs and LSTMs are (super-easy and super-complicated, respectively) ways of accomplishing this. 
- CLLMs exactly follow Shannon: they extract an $n$-th order transition matrix from a source of natural text, and generate new text by simply applying the transition matrix one $n$-gram at a time.
- LSTMs apply the idea of states and transitions (i.e. Markov chains) to neural networks.

![](img/rnn.jpg)